# Advanced Querying Mongo

Importing libraries and setting up connection

In [30]:
# from pymongo import MongoClient
# client = MongoClient()

from pymongo import MongoClient
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()
cursor = db.list_collections()
from pymongo import ASCENDING, DESCENDING


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [31]:
cur = ({"name":{"$eq":"Babelgum"}})

list(db['companies'].find(cur,{"_id":0,"name":1}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [59]:
cur = ({"number_of_employees" : {"$gt": 5000}})

list(db["companies"].find(cur,{"_id":0,"name":1}).limit(20).sort([("number_of_employees",DESCENDING)]))

[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'},
 {'name': 'Sony'},
 {'name': 'LG'},
 {'name': 'Ford'},
 {'name': 'Boeing'},
 {'name': 'Digital Equipment Corporation'},
 {'name': 'Nokia'},
 {'name': 'MItsubishi Electric'},
 {'name': 'MItsubishi Electric'},
 {'name': 'Comcast'},
 {'name': 'Bertelsmann'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [69]:
cur = ({"founded_year":{"$in":list(range(2000,2006))}})

list(db["companies"].find(cur,{"_id":0,"name":1, "founded_year":1}))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Pando Networks', 'founded_year': 2004},
 {'name': 'Ikan', 'foun

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [123]:
cur = ({"$and":[{"founded_year":{"$lt":2010}}, 
               {"ipo.valuation_amount":{"$gt":100000000}}]})

list(db["companies"].find(cur,{"_id":0,"name":1, "ipo":1}))

[{'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Brightcove',
  'ipo': {'valuation_amount': 290000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 2,
   'pub_day': 17,
   'stock_symbol': 'NASDAQ:BCOV'}},
 {'name': 'Amazon',
  'ipo': {'valu

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [109]:
cur = ({"$and":[{"number_of_employees":{"$lt":1000}}, 
               {"founded_year":{"$lt":2005}}]})

list(db["companies"].find(cur,{"_id":0,"name":1}).limit(10).sort([("number_of_employees",DESCENDING)]))

[{'name': 'Infinera Corporation'},
 {'name': 'NorthPoint Communications Group'},
 {'name': '888 Holdings'},
 {'name': 'Forrester Research'},
 {'name': 'SonicWALL'},
 {'name': 'Webmetrics'},
 {'name': 'Cornerstone OnDemand'},
 {'name': 'Yelp'},
 {'name': 'ZoomInfo'},
 {'name': 'MySpace'}]

### 6. All the companies that don't include the `partners` field.

In [207]:
cur = ({"partners":{"$exists":False}})

list(db["companies"].find(cur,{"_id":0,"name":1}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [145]:
cur = ({"category_code":{"$type":"null"}})

list(db['companies'].find(cur,{"_id":0,"name":1}))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'MMDAYS'},
 {'name': 'Touch Clarity'},
 {'name': 'Inside Group'},
 {'name': 'Repeater Store'},
 {'name': 'NewPersia'},
 {'name': 'Tapesh'},
 {'name': 'iPersians'},
 {'name': 'Pyra Labs'},
 {'name': 'Feedmap'},
 {'name': 'NuvoMedia'},
 {'name': 'Intwine'},
 {'name': 'The Weinstein Company'},
 {'name': 'ExecuNet'},
 {'name': 'A1Vacations'},
 {'name': 'TripHomes'},
 {'name': 'Picturehouse'},
 {'name': 'Covalent Technologies'},
 {'name': 'Epic Trip'},
 {'name': 'Telepost'},
 {'name': 'Fidelity'},
 {'name': 'Quirk'},
 {'name': 'Skyrove'},
 {'name': 'Leechblock'},
 {'name': 'My Success Gateway'},
 {'name': 'The Easynet Group'},
 {'name': 'RealNames'},
 {'name': 'NetNames'},
 {'name': 'Sniperoo'},
 {'name': 'JD Project'},
 {'name': 'ABC News'},
 {'name': 'Teoma Technologies'},
 {'name': 'cometro'},
 {'name': 'TechJuicer'},
 

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [121]:
cur = ({"number_of_employees":{"$in":list(range(100,1001))}})

list(db["companies"].find(cur,{"_id":0,"name":1, "number_of_employees":1}))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [124]:
cur = ({"ipo.valuation_amount" : {"$gt": 0}})

list(db["companies"].find(cur,{"_id":0,"name":1}).sort([("ipo.valuation_amount",DESCENDING)]))

[{'name': 'GREE'},
 {'name': 'Facebook'},
 {'name': 'Amazon'},
 {'name': 'Twitter'},
 {'name': 'Groupon'},
 {'name': 'Tencent'},
 {'name': 'Western Digital'},
 {'name': 'LinkedIn'},
 {'name': 'BMC Software'},
 {'name': 'Rackspace'},
 {'name': 'Baidu'},
 {'name': 'TripAdvisor'},
 {'name': 'HomeAway'},
 {'name': 'Zillow'},
 {'name': 'Nielsen'},
 {'name': 'Yelp'},
 {'name': 'Chegg'},
 {'name': 'Chegg'},
 {'name': 'RPX Corporation'},
 {'name': 'Higher One'},
 {'name': 'OpenTable'},
 {'name': 'Jive Software'},
 {'name': 'BrightSource Energy'},
 {'name': 'QlikTech'},
 {'name': 'Millennial Media'},
 {'name': 'Mixi'},
 {'name': 'Cornerstone OnDemand'},
 {'name': 'Infoblox'},
 {'name': 'SolarWinds'},
 {'name': 'Wix'},
 {'name': 'Marketo'},
 {'name': 'Opsware'},
 {'name': 'Emdeon'},
 {'name': 'Shutterfly'},
 {'name': 'Telenav'},
 {'name': 'Rally Software'},
 {'name': 'STR'},
 {'name': 'Brightcove'},
 {'name': 'ChannelAdvisor'},
 {'name': 'Elster Group'},
 {'name': 'DemandTec'},
 {'name': 'KIT di

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [136]:
cur = ({"number_of_employees" : {"$gt": 0}})

list(db["companies"].find(cur,{"_id":0,"name":1, "number_of_employees":1}).limit(10).sort([("number_of_employees",DESCENDING)]))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [147]:
cur = ({"founded_month" : {"$in": list(range(7,13))}})

list(db["companies"].find(cur,{"_id":0,"name":1}).limit(1000))

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Omnidrive'},
 {'name': 'Digg'},
 {'name': 'Joost'},
 {'name': 'Plaxo'},
 {'name': 'eBay'},
 {'name': 'Powerset'},
 {'name': 'Kyte'},
 {'name': 'Thoof'},
 {'name': 'Jingle Networks'},
 {'name': 'Wesabe'},
 {'name': 'SmugMug'},
 {'name': 'LifeLock'},
 {'name': 'Skype'},
 {'name': 'Pando Networks'},
 {'name': 'Ikan'},
 {'name': 'delicious'},
 {'name': 'AllPeers'},
 {'name': 'Wize'},
 {'name': 'SellABand'},
 {'name': 'iContact'},
 {'name': 'MeeVee'},
 {'name': 'blinkx'},
 {'name': 'Zlio'},
 {'name': 'Jaiku'},
 {'name': 'Yelp'},
 {'name': 'Yapta'},
 {'name': 'Fleck'},
 {'name': 'SideStep'},
 {'name': 'Instructables'},
 {'name': 'RockYou'},
 {'name': 'Swivel'},
 {'name': 'Netvibes'},
 {'name': 'Slide'},
 {'name': 'Google'},
 {'name': 'CrowdVine'},
 {'name': 'TheFind'},
 {'name': 'Sway'},
 {'name': 'CornerWorld'},
 {'name': 'FlickIM'},
 {'name': 'Terabitz'},
 {'name': 'Kaboodle'},
 {'name': 'Go2Web20'},
 {'name': 'Redfin'},
 {'name': 'Wink'

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [158]:
cur = ({"$and":[{"founded_year":{"$lt":2000}}, 
               {"acquisition.price_amount":{"$gt":1000000}}]})

list(db["companies"].find(cur,{"_id":0,"name":1,"acquisition.price_amount":1 }))

[{'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep', 'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar', 'acquisition': {'price_amount': 25000000}},
 {'name': 'Cyworld', 'acquisition': {'price_amount': 7140000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish', 'acquisition': {'price_amount': 300000000}},
 {'name': 'Neopets', 'acquisition': {'price_amount': 160000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Sun Microsystems', 'acquisition': {'price_amount': 7400000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Sabre', 'acquisition': {'price_amount': 4300000000}},
 {'name': 'Lastminute', 'acquisition': {'price_amount': 1008000000}},
 {'name': 'Sportsline', 'acquisition': {'price_amount': 64000000}},
 {'name': 'Webshots', 'acquisition': {'price_amount': 71000000}},
 {'name': 'MIVA', 'acquisition': {'price_amount': 11600000

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [192]:
cur = ({"acquisition.acquired_year" : {"$gt": 2010}})

list(db["companies"].find(cur,{"_id":0,"name":1, "acquisiton.acquired_year":1}).limit(20).sort([("acquisiton.price_amount",DESCENDING)]))


[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'Kyte'},
 {'name': 'Jingle Networks'},
 {'name': 'blogTV'},
 {'name': 'delicious'},
 {'name': 'Revision3'},
 {'name': 'iContact'},
 {'name': 'Mashery'},
 {'name': 'Dailymotion'},
 {'name': 'KickApps'},
 {'name': 'Instructables'},
 {'name': 'Netvibes'},
 {'name': 'Dogster'},
 {'name': 'adBrite'},
 {'name': 'Meebo'},
 {'name': 'Seesmic'},
 {'name': 'Zvents'},
 {'name': 'Oodle'}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [169]:
cur = ({"founded_year" : {"$exists": True}})

list(db["companies"].find(cur,{"_id":0,"name":1, "founded_year":1}).limit(20).sort([("founded_year",DESCENDING)]))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'Tongxue', 'founded_year': 2013},
 {'name': 'Shopseen', 'founded_year': 2013},
 {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
 {'name': 'PeekYou', 'founded_year': 2012},
 {'name': 'headr', 'founded_year': 2012},
 {'name': 'Pinger', 'founded_year': 2012},
 {'name': 'Widgetbox', 'founded_year': 2012},
 {'name': 'Mobiluck', 'founded_year': 2012},
 {'name': 'Skydeck', 'founded_year': 2012},
 {'name': 'Simplicant', 'founded_year': 2012}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [179]:
cur = ({"founded_day" : {"$in": list(range(1,8))}})

list(db["companies"].find(cur,{"_id":0,"name":1, "founded_day":1}).limit(10).sort([("acquisition.price_amount", DESCENDING)]))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'BlueLithium', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [183]:
cur = ({"$and":[{"category_code":{"$eq":"web"}}, 
               {"number_of_employees":{"$gt":4000}}]})

list(db["companies"].find(cur,{"_id":0,"name":1}).sort([("number_of_employees", ASCENDING)]))

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Rakuten'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Groupon'},
 {'name': 'Yahoo!'},
 {'name': 'eBay'},
 {'name': 'Experian'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [185]:
cur = ({"$and":[{"acquisition.price_currency_code":{"$eq":"EUR"}}, 
               {"acquisition.price_amount":{"$gt":10000000}}]})

list(db["companies"].find(cur,{"_id":0,"name":1}))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [189]:
cur = ({"acquisition.acquired_month":{"$in": list(range(1,4))}})

list(db["companies"].find(cur,{"_id":0,"name":1, "acquired_month":1}).limit(10))

[{'name': 'Kyte'},
 {'name': 'NetRatings'},
 {'name': 'blogTV'},
 {'name': 'Livestream'},
 {'name': 'iContact'},
 {'name': 'Coghead'},
 {'name': 'Dailymotion'},
 {'name': 'Netvibes'},
 {'name': 'Flickr'},
 {'name': 'BabyCenter'}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [208]:
cur = ({"$and":[{"founded_year":{"$in":list(range(2000,2011))}}, 
               {"acquisition.acquired_year":{"$gte":2011}}]})

list(db["companies"].find(cur,{"_id":0,"name":1}))

# se que he hecho "trampa" al no usar el not :(

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'Kyte'},
 {'name': 'Jingle Networks'},
 {'name': 'blogTV'},
 {'name': 'delicious'},
 {'name': 'Revision3'},
 {'name': 'iContact'},
 {'name': 'Mashery'},
 {'name': 'Dailymotion'},
 {'name': 'KickApps'},
 {'name': 'Instructables'},
 {'name': 'Netvibes'},
 {'name': 'Dogster'},
 {'name': 'adBrite'},
 {'name': 'Meebo'},
 {'name': 'Seesmic'},
 {'name': 'Zvents'},
 {'name': 'Oodle'},
 {'name': 'Flixster'},
 {'name': 'IceRocket'},
 {'name': 'hi5'},
 {'name': 'Indeed'},
 {'name': 'Grockit'},
 {'name': 'Zenfolio'},
 {'name': 'Wishpot'},
 {'name': 'Retrevo'},
 {'name': 'SelectMinds'},
 {'name': 'Flock'},
 {'name': 'TimeBridge'},
 {'name': 'TokBox'},
 {'name': 'Six Apart'},
 {'name': 'Doostang'},
 {'name': 'MySpace'},
 {'name': 'Vlingo'},
 {'name': 'Docstoc'},
 {'name': 'sezmi'},
 {'name': 'Greystripe'},
 {'name': 'LocaModa'},
 {'name': 'Thumbplay'},
 {'name': 'OMGPOP'},
 {'name': 'Daylife'},
 {'name': 'Behance'},
 {'name': 'At

### 20. All the companies that have been 'deadpooled' after the third year.

In [214]:
cur = ({"deadpooled_year":{"$gt": 3}})

list(db["companies"].find(cur,{"_id":0,"name":1, "founded_year":1, "deadpooled_year":1}).limit(10))

[{'name': 'Omnidrive', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'Babelgum', 'founded_year': 2007, 'deadpooled_year': 2013},
 {'name': 'Sparter', 'founded_year': 2007, 'deadpooled_year': 2008},
 {'name': 'Thoof', 'founded_year': 2006, 'deadpooled_year': 2013},
 {'name': 'Mercora', 'founded_year': None, 'deadpooled_year': 2008},
 {'name': 'Wesabe', 'founded_year': 2005, 'deadpooled_year': 2010},
 {'name': 'Jangl SMS', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'Stickam', 'founded_year': 2006, 'deadpooled_year': 2013},
 {'name': 'AllPeers', 'founded_year': 2004, 'deadpooled_year': 2008},
 {'name': 'AllofMP3', 'founded_year': 2000, 'deadpooled_year': 2007}]

In [ ]:
# Esta claramente mal porque de 2007-2008 no hay mas de 3 años (Sparter)